In [ ]:
!pip install --upgrade pip
!pip install torch torchaudio transformers datasets datasets[audio] accelerate evaluate jiwer huggingface_hub yt-dlp tqdm librosa soundfile wandb
!pip install -q -U bitsandbytes

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import os
from datasets import load_from_disk
from torch.utils.data import DataLoader
from transformers import WhisperProcessor, AutoTokenizer, AutoModelForCausalLM, WhisperModel
import yt_dlp
import torch
import torch.nn as nn
import torchaudio
from datasets import load_dataset
from transformers import WhisperFeatureExtractor, WhisperForConditionalGeneration, BitsAndBytesConfig
from datasets import Audio
from torch.nn.utils.rnn import pad_sequence


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
#Nice utility functions 
def text_2_ids_and_attention_mask(tokenizer, input_txt, truncate=False):
    txt = input_txt
    res = tokenizer(txt, return_tensors="pt")

    if truncate:
        return res.input_ids[:, 1:], res.attention_mask[:, 1:]

    return res.input_ids, res.attention_mask
def prompt_template_fn(prompt="Describe the sound of the given file"):
    system_message = "You are a helpful AI who follows instruction carefully"

    #mistral's prompt template
    # prompt_prefix = f"""<|im_start|>system
    # {system_message}<|im_end|>
    # <|im_start|>user
    # {prompt}"""

    #Llama-3 prompt template
    # https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
    prompt_prefix = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    {system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>
    {prompt}
    """
    return prompt_prefix




def end_template():
    ###Mistral end token
    # return """
    # <|im_end|>
    # <|im_start|>assistant
    # """

    #llama-3 end tokens
    return """
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """

In [6]:
ds = load_dataset("mahendra0203/musiccaps_processed_full" )
main_dataset = ds['train'].train_test_split(test_size=0.1)

train_val_split_dataset = main_dataset['train'].train_test_split(test_size=0.1)

train_dataset = train_val_split_dataset['train']
val_dataset = train_val_split_dataset['test']

test_dataset = main_dataset['test']

train_dataset = train_dataset.cast_column('audio', Audio(sampling_rate= 16000))
val_dataset = val_dataset.cast_column('audio', Audio(sampling_rate= 16000))
test_dataset = test_dataset.cast_column('audio', Audio(sampling_rate= 16000))

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Generating train split:   0%|          | 0/5355 [00:00<?, ? examples/s]

Train dataset size: 4337
Validation dataset size: 482
Test dataset size: 536


In [7]:

# train_dataset = load_from_disk(os.path.join('/home', "dataset/train/train_musiccaps_audio_data"))
# val_dataset = load_from_disk(os.path.join('/home', "dataset/val/val_musiccaps_audio_data"))

#START --This is the Rgiht code, testing with a small dataset noe
# ds = load_dataset("mahendra0203/musiccaps_processed_full" )
# main_dataset = ds['train'].train_test_split(test_size=0.1)
#END--

# ds = load_dataset("mahendra0203/musiccaps_processed" )
# small_dataset = ds['train'].train_test_split(test_size=0.05)
# main_dataset = small_dataset['test'].train_test_split(test_size = 0.1)

# # train_dataset = load_dataset("mahendra0203/musiccaps_processed")
# # main_dataset = load_dataset("mahendra0203/musiccaps_processed")
# train_dataset = main_dataset['train']
# val_dataset = main_dataset['test']

# # #cast audio to sample rate of 16000
# train_dataset = train_dataset.cast_column('audio', Audio(sampling_rate= 16000))
# val_dataset = val_dataset.cast_column('audio', Audio(sampling_rate= 16000))

# print(f"Train dataset size: {len(train_dataset)}")
# print(f"Validation dataset size: {len(val_dataset)}")

In [7]:
test_dataset[0]

{'audio': {'path': 'yt_id_vy2gUzgmBzQ_180_190.wav',
  'array': array([-0.05502319,  0.56330872,  0.58569336, ...,  0.32862854,
         -0.21427917, -0.25509644]),
  'sampling_rate': 16000},
 'caption': 'This goth metal song features a male voice singing the main melody. The voice is deep. Toward the end, there are other male voices singing vocables. This is accompanied by heavy percussion using a five-beat common time pattern played on floor toms with a shuffle note each note creating a triplet feel. The distortion guitars play heavy descending chord riffs using palm muting technique. The bass guitar plays the root notes of the chords. At the background, there is an high pitched sound played on a synth that gives this song an eerie feel. This song can be played at the end credits of a horror movie.',
 'youtube_id': 'vy2gUzgmBzQ',
 'start_time': 180,
 'end_time': 190,
 'aspect_list': "['heavy metal', 'goth metal', 'male voice', 'amateur recording', 'distortion guitars', 'heavy percussi

In [8]:
whisper_model_name = "openai/whisper-large-v2"
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper_model_name)
whisper_processor = WhisperProcessor.from_pretrained(whisper_model_name)
#Download the llm tokenizer.LLM will be downloaded in the model
# model_name = "facebook/opt-1.3b"
model_name= "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# llm = AutoModelForCausalLM.from_pretrained(model_name)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# bnb_config = BitsAndBytesConfig(
#             load_in_4bit=True,
#             bnb_4bit_use_double_quant=True,
#             bnb_4bit_quant_type="nf4",
#             bnb_4bit_compute_dtype=torch.bfloat16
#         )
# llm = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

In [9]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz

    audio = batch["audio"]
    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids 
    # batch["caption"] = batch["caption"]
    token_result = tokenizer(batch["caption"])
    batch["caption_ids"] = token_result.input_ids
    batch["caption_attention_mask"] = token_result.attention_mask

    # Encode prompt
    prompt_tokens = tokenizer(prompt_template_fn())
    batch["prompt_ids"] = prompt_tokens.input_ids
    batch["prompt_attention_mask"] = prompt_tokens.attention_mask
    
    # Encode end prompt
    end_prompt_tokens = tokenizer(end_template())
    batch["end_prompt_ids"] = end_prompt_tokens.input_ids
    batch["end_prompt_attention_mask"] = end_prompt_tokens.attention_mask

    return batch

In [10]:

new_train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, num_proc=64)
new_val_dataset = val_dataset.map(prepare_dataset, remove_columns=val_dataset.column_names, num_proc=64)
new_test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, num_proc=64)


Map (num_proc=64):   0%|          | 0/4337 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/482 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/536 [00:00<?, ? examples/s]

In [11]:
new_train_dataset.save_to_disk('/home/mapped_dataset/train/')

Saving the dataset (0/9 shards):   0%|          | 0/4337 [00:00<?, ? examples/s]

In [12]:
new_val_dataset.save_to_disk('/home/mapped_dataset/val/')

Saving the dataset (0/1 shards):   0%|          | 0/482 [00:00<?, ? examples/s]

In [13]:
new_test_dataset.save_to_disk('/home/mapped_dataset/test/')

Saving the dataset (0/2 shards):   0%|          | 0/536 [00:00<?, ? examples/s]

In [16]:
# new_train_dataset=None
# new_val_dataset=None
# new_test_dataset=None

In [17]:
# new_train_dataset = load_from_disk('/home/mapped_dataset/train/')
# new_val_dataset = load_from_disk('/home/mapped_dataset/val/')
# new_test_dataset = load_from_disk('/home/mapped_dataset/test/')

In [14]:
# Set batch size
def collate_fn(batch):
    # Separate different components of the batch
    input_features = [whisper_processor.feature_extractor.pad({"input_features": item['input_features']}, return_tensors="pt").input_features for item in batch]
    # formatted_features = processor.feature_extractor.pad({"input_features": input_features}, return_tensors="pt")

    caption_ids = [item['caption_ids'] for item in batch]
    caption_attention_mask = [item['caption_attention_mask'] for item in batch]
    prompt_ids = [item['prompt_ids'] for item in batch]
    prompt_attention_mask = [item['prompt_attention_mask'] for item in batch]
    end_prompt_ids = [item['end_prompt_ids'] for item in batch]
    end_prompt_attention_mask = [item['end_prompt_attention_mask'] for item in batch]

    # # Pad sequences
    #128009 is end of message token -> <|eot_id|> for llama3
    caption_ids = pad_sequence([torch.tensor(x) for x in caption_ids], batch_first=True, padding_value=128009)
    caption_attention_mask = pad_sequence([torch.tensor(x) for x in caption_attention_mask], batch_first=True, padding_value=128009)
    prompt_ids = pad_sequence([torch.tensor(x) for x in prompt_ids], batch_first=True, padding_value=128009)
    prompt_attention_mask = pad_sequence([torch.tensor(x) for x in prompt_attention_mask], batch_first=True, padding_value=128009)
    end_prompt_ids = pad_sequence([torch.tensor(x) for x in end_prompt_ids], batch_first=True, padding_value=128009)
    end_prompt_attention_mask = pad_sequence([torch.tensor(x) for x in end_prompt_attention_mask], batch_first=True, padding_value=128009)

    # caption_ids = pad_sequence([torch.tensor(x) for x in caption_ids], batch_first=True, padding_value=0)
    # caption_attention_mask = pad_sequence([torch.tensor(x) for x in caption_attention_mask], batch_first=True, padding_value=0)
    # prompt_ids = pad_sequence([torch.tensor(x) for x in prompt_ids], batch_first=True, padding_value=0)
    # prompt_attention_mask = pad_sequence([torch.tensor(x) for x in prompt_attention_mask], batch_first=True, padding_value=0)
    # end_prompt_ids = pad_sequence([torch.tensor(x) for x in end_prompt_ids], batch_first=True, padding_value=0)
    # end_prompt_attention_mask = pad_sequence([torch.tensor(x) for x in end_prompt_attention_mask], batch_first=True, padding_value=0)
    
    # # Stack input features (assuming they're already of the same size)
    input_features = torch.stack([torch.tensor(x) for x in input_features])

    return {
        'input_features': input_features,
        'caption_ids': caption_ids,
        'caption_attention_mask': caption_attention_mask,
        'prompt_ids': prompt_ids,
        'prompt_attention_mask': prompt_attention_mask,
        'end_prompt_ids': end_prompt_ids,
        'end_prompt_attention_mask': end_prompt_attention_mask
    }
batch_size = 8  # Adjust this value based on your GPU memory and model size

# Create DataLoaders
train_dataloader = DataLoader(new_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(new_val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(new_test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

# Verify the batches
for batch in train_dataloader:
    print("Input features shape:", batch['input_features'].shape)
    print("Caption IDs shape:", batch['caption_ids'].shape)
    print("Caption attention mask shape:", batch['caption_attention_mask'].shape)
    print("Prompt IDs shape:", batch['prompt_ids'].shape)
    print("Prompt attention mask shape:", batch['prompt_attention_mask'].shape)
    print("End prompt IDs shape:", batch['end_prompt_ids'].shape)
    print("End prompt attention mask shape:", batch['end_prompt_attention_mask'].shape)
    # print('\n'.join(tokenizer.batch_decode(batch['caption_ids'][-10: ])))

    break

print("DataLoader creation completed.")

Input features shape: torch.Size([8, 80, 3000])
Caption IDs shape: torch.Size([8, 76])
Caption attention mask shape: torch.Size([8, 76])
Prompt IDs shape: torch.Size([8, 31])
Prompt attention mask shape: torch.Size([8, 31])
End prompt IDs shape: torch.Size([8, 9])
End prompt attention mask shape: torch.Size([8, 9])
DataLoader creation completed.


/tmp/ipykernel_146/3557587238.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_features = torch.stack([torch.tensor(x) for x in input_features])


In [19]:
# tmp_batch = next(iter(train_dataloader))

In [20]:
# dataloader_iter = iter(train_dataloader)
# batch = next(dataloader_iter)
# batch.keys()

# len(batch['input_features'][0])

In [21]:
# tokenizer.pad_token_id = tokenizer.eos_token_id
# tokenizer.pad_token = tokenizer.eos_token

In [22]:
# tokenizer.encode('<|eot_id|>',  padding=True)

tokenizer

In [15]:
class TunableWhisperProjection(nn.Module):
    def __init__(self, input_embedding_size=1280, output_embedding_size=2048):
        super().__init__()

        self.pool = nn.AdaptiveAvgPool1d(250)
        self.proj = nn.Linear(input_embedding_size, output_embedding_size, bias=False)
        self.ln1 = nn.LayerNorm(input_embedding_size)

    def forward(self, whisper_output):
        # Assuming whisper_output is of shape (batch_size, seq_len, input_embedding_size)
        # Transpose for adaptive pooling
        pooled = self.pool(whisper_output.transpose(-2, -1))
        
        # Transpose back and apply layer norm
        normalized = self.ln1(pooled.transpose(-2, -1))
        
        # Project to output embedding size
        projected = self.proj(normalized)
        
        return projected

In [16]:
class MultiModalLLM(nn.Module):
    def __init__(self, whisper_model_name="openai/whisper-large-v2", llm_model_name="facebook/opt-1.3b"):
        super().__init__()
        # Audio encoder (Whisper)
        bnb_whsiper_config = BitsAndBytesConfig(
            load_in_8bit=True,
        )
        
        self.audio_encoder = WhisperModel.from_pretrained(whisper_model_name).get_encoder()
        # Freeze Whisper parameters
        for param in self.audio_encoder.parameters():
            param.requires_grad = False
        # Language Model
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        self.llm = AutoModelForCausalLM.from_pretrained(llm_model_name, quantization_config=bnb_config)
        for param in self.llm.parameters():
            param.requires_grad = False
        self.projection = TunableWhisperProjection(input_embedding_size=1280, output_embedding_size=4096)

    def forward(self, input_features, prompt_ids, prompt_attention_mask, 
                end_prompt_ids, end_prompt_attention_mask, 
                caption_ids, caption_attention_mask):
        # print the shape of all these above
        # print(f"input_features.shape: {input_features.shape}")
        # print(f"prompt_ids.shape: {prompt_ids.shape}")
        # print(f"prompt_attention_mask.shape: {prompt_attention_mask.shape}")
        # print(f"end_prompt_ids.shape: {end_prompt_ids.shape}")
        # print(f"end_prompt_attention_mask.shape: {end_prompt_attention_mask.shape}")
        # print(f"caption_ids.shape: {caption_ids.shape}")
        # print(f"caption_attention_mask.shape: {caption_attention_mask.shape}")
        
        # Process audio with Whisper encoder
        audio_outputs = self.audio_encoder.forward(input_features).last_hidden_state
        # print(f"audio_outputs.shape: {audio_outputs.shape}")
        projected_audio = self.projection(audio_outputs)
        # print(f"projected_audio.shape: {projected_audio.shape}")
        # caption_ids = caption_ids.to(device)
        # prompt_ids = prompt_ids.to(device)
        # end_prompt_ids = end_prompt_ids.to(device)

        if(self.llm.model.name_or_path == "meta-llama/Meta-Llama-3-8B-Instruct"):
            cap_embeds = self.llm.model.embed_tokens(caption_ids)
            prompt_embeds = self.llm.model.embed_tokens(prompt_ids)
            end_prompt_embeds = self.llm.model.embed_tokens(end_prompt_ids)
        else:
            cap_embeds = self.llm.model.get_decoder().embed_tokens(caption_ids)
            prompt_embeds = self.llm.model.get_decoder().embed_tokens(prompt_ids)
            end_prompt_embeds = self.llm.model.get_decoder().embed_tokens(end_prompt_ids)

        # print(f"cap_embeds.shape: {cap_embeds.shape}")
        # print(f"prompt_embeds.shape: {prompt_embeds.shape}")
        # print(f"end_prompt_embeds.shape: {end_prompt_embeds.shape}")
        
        bs, audio_seq = projected_audio.shape[:2]

        # print(f"batch_size: {bs}")
        # print(f"audio_seq: {audio_seq}")
        
        inputs_embeds = torch.concat(
            (
                prompt_embeds,
                projected_audio.to(cap_embeds.dtype),
                end_prompt_embeds,
                cap_embeds,
            ),
            dim=1,
        )
        # print(f"inputs_embeds.shape: {inputs_embeds.shape}")
        
        attention_mask = torch.concat(
            (
                prompt_attention_mask,
                torch.ones(bs, audio_seq).to(caption_ids.device),
                end_prompt_attention_mask,
                caption_attention_mask,
            ),
            dim=1,
        )
        # print(f"attention_mask.shape: {attention_mask.shape}")
        
        outputs = self.llm(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask
        )
        return outputs, audio_seq
        

In [17]:
import wandb

In [18]:
wandb.login(key="0bce4133f74d94dae8633922fa2e8760e280f639")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/.netrc


True

In [23]:
def move_model_to_gpu(model):
    if torch.cuda.is_available():
        model = model.cuda()
        for param in model.parameters():
            param.data = param.data.cuda()
            if param.grad is not None:
                param.grad.data = param.grad.data.cuda()
    return model
    
def move_to_device(batch, device):
    return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}

In [24]:
def log_results_to_wandb(results):
    table = wandb.Table(columns=["Ground Truth", "Generated"])
    for result in results:
        table.add_data(result["Ground Truth"], result["Generated"])
    wandb.log({"Caption Comparison": table})
    
def run_inference(model, dataloader, tokenizer, device, num_samples=10):
    model.eval()
    results = []
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader, desc="Running inference")):
            if i >= num_samples:
                break
            
            batch = move_to_device(batch, device)
            
            outputs, audio_len = model(**batch)
            prompt_ids = batch['prompt_ids']
            end_prompt_ids = batch['end_prompt_ids']
            caption_ids = batch['caption_ids']
            
            prompt_ids_seq = prompt_ids.shape[1]
            end_prompt_ids_seq = end_prompt_ids.shape[1]
            audio_seq = audio_len
            logits_start = prompt_ids_seq + audio_seq + end_prompt_ids_seq
            
            logits = outputs.logits
            op_logits = logits[:, logits_start:-1, :].contiguous()
            caption_labels = caption_ids[:, 1:].contiguous()
            
            sampled = torch.multinomial(op_logits[:, -1, :].softmax(dim=-1), caption_labels.shape[1])
            
            ground_truth = tokenizer.batch_decode(batch['caption_ids'], skip_special_tokens=True)
            generated = tokenizer.batch_decode(sampled,skip_special_tokens=True)
            
            for gt, gen in zip(ground_truth, generated):
                results.append({"Ground Truth": gt, "Generated": gen})
    
    return results

In [29]:
import torch
from tqdm import tqdm
import torch.nn.functional as F
import wandb

def calculate_loss(outputs, batch, audio_seq):
    logits = outputs.logits
    prompt_ids = batch['prompt_ids']
    end_prompt_ids = batch['end_prompt_ids']
    caption_ids = batch['caption_ids']
    # proj_op = outputs.projected_audio  # Assuming your model returns this
    # print(prompt_ids.shape)
    # print(end_prompt_ids.shape)
    # print(caption_ids.shape)
    # print(audio_seq)
    
    prompt_ids_seq = prompt_ids.shape[1]
    end_prompt_ids_seq = end_prompt_ids.shape[1]
    audio_seq = audio_seq
    logits_start = prompt_ids_seq + audio_seq + end_prompt_ids_seq
    # print(f"Logits shape: {logits.shape}")
    # print(f"Logits start: {logits_start}")
    # print(f"Caption IDs shape: {caption_ids.shape}")

    # Ensure logits_start is not out of bounds
    # if logits_start >= logits.shape[1]:
    #     raise ValueError(f"logits_start ({logits_start}) is out of bounds for logits shape {logits.shape}")

    op_logits = logits[:, logits_start:-1, :].contiguous()
    caption_labels = caption_ids[:, 1:].contiguous()
    
    # print(f"op_logits shape: {op_logits.shape}")
    # print(f"caption_labels shape: {caption_labels.shape}")
    
    if op_logits.shape[1] != caption_labels.shape[1]:
        raise ValueError(f"Shape mismatch: op_logits {op_logits.shape}, caption_labels {caption_labels.shape}")

    
    loss = F.cross_entropy(
        op_logits.view(-1, op_logits.shape[-1]), caption_labels.view(-1)
    )
    return loss
    

    
def train(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device, wandb_proj_name="MultiModal LLM", wandb_run_name="Training Run"):
    wandb.init(project=wandb_proj_name, name=wandb_run_name)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    wandb.config.update({
        "learning_rate": learning_rate,
        "epochs": num_epochs,
        "batch_size": train_dataloader.batch_size,
        "model_name": type(model).__name__,
        # Add any other hyperparameters you want to track
    })
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training
        model.train()
        total_train_loss = 0
        for batch in tqdm(train_dataloader, desc="Training"):
            # batch = {k: v.to(device) for k, v in batch.items()}
            batch = move_to_device(batch, device)
            
            optimizer.zero_grad()
            outputs, audio_seq = model(**batch)
            loss = calculate_loss(outputs, batch, audio_seq)
            # loss = outputs.loss
            # print(f"loss:{loss.item()}")
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            # break;
            
        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Average Training Loss: {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validating"):
                batch = move_to_device(batch, device)
                # batch = {k: v.to(device) for k, v in batch.items()}
                outputs, audio_seq = model(**batch)
                # loss = outputs.loss
                # outputs = model(**batch)
                loss = calculate_loss(outputs, batch, audio_seq)
                total_val_loss += loss.item()
                
        avg_val_loss = total_val_loss / len(val_dataloader)
        print(f"Average Validation Loss: {avg_val_loss:.4f}")
        print()
        
        # Log metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss
        })
        # Run inference on a batch and log results
        inference_results = run_inference(model, test_dataloader, tokenizer, device, num_samples=8)
        log_results_to_wandb(inference_results)
        # Optionally, log model parameters
        wandb.watch(model)

    torch.save(model.state_dict(), "final_model.pth")
    wandb.save("final_model.pth")

    # Finish the wandb run
    wandb.finish()


# Setup
# model = MultimodalAudioTextModel()  # Make sure this is defined
model = MultiModalLLM(whisper_model_name, model_name)
model = move_model_to_gpu(model)
num_epochs = 10
learning_rate = 1.5e-3

# Assuming train_dataloader and val_dataloader are already created
train(model, train_dataloader, val_dataloader, num_epochs, learning_rate, device,wandb_proj_name="MultiLLM - whisper-large-v2 LLAMA 3-8b Instruct",  wandb_run_name="epoch_10_lr_1.5e-3 -- padding fixes")



`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1/10


Training:   0%|          | 0/543 [00:00<?, ?it/s]/tmp/ipykernel_33161/3557587238.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_features = torch.stack([torch.tensor(x) for x in input_features])
Training: 100%|██████████| 543/543 [34:53<00:00,  3.86s/it]


Average Training Loss: 4.9006


...

Average Training Loss: 3.2535


Validating:   0%|          | 0/61 [00:02<?, ?it/s]


Average Validation Loss: 0.0538



Running inference:  12%|█▏        | 8/67 [00:23<02:55,  2.98s/it]


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▄▃▂▂▂▁▁▁▁
val_loss,▆▄▇▂▅▁▄▅█▃
epoch,10
train_loss,3.25346
val_loss,0.05377


In [42]:
# model = model.cpu()
    
# # Delete the model
# del model

# torch.cuda.empty_cache()
    
# # Force garbage collection
# import gc
# gc.collect()

13921

In [31]:
# def log_results_to_wandb(results):
#     table = wandb.Table(columns=["Ground Truth", "Generated"])
#     for result in results:
#         table.add_data(result["Ground Truth"], result["Generated"])
#     wandb.log({"Caption Comparison": table})
    
# def run_inference(model, dataloader, tokenizer, device, num_samples=10):
#     model.eval()
#     results = []
    
#     with torch.no_grad():
#         for i, batch in enumerate(tqdm(dataloader, desc="Running inference")):
#             if i >= num_samples:
#                 break
            
#             batch = move_to_device(batch, device)
            
#             outputs, audio_len = model(**batch)
#             prompt_ids = batch['prompt_ids']
#             end_prompt_ids = batch['end_prompt_ids']
#             caption_ids = batch['caption_ids']
            
#             prompt_ids_seq = prompt_ids.shape[1]
#             end_prompt_ids_seq = end_prompt_ids.shape[1]
#             audio_seq = audio_len
#             logits_start = prompt_ids_seq + audio_seq + end_prompt_ids_seq
            
#             logits = outputs.logits
#             op_logits = logits[:, logits_start:-1, :].contiguous()
#             caption_labels = caption_ids[:, 1:].contiguous()
            
#             sampled = torch.multinomial(op_logits[:, -1, :].softmax(dim=-1), caption_labels.shape[1])
            
#             ground_truth = tokenizer.batch_decode(batch['caption_ids'])
#             generated = tokenizer.batch_decode(sampled)
            
#             for gt, gen in zip(ground_truth, generated):
#                 results.append({"Ground Truth": gt, "Generated": gen})
    
#     return results

In [32]:
# run_inference(model, val_dataloader, tokenizer, device, 2)

In [33]:
# #run inference

# def run_inference(model, dataloader, tokenizer, sample, device):
#     model.eval()
#     with torch.no_grad():
#         new_batch = next(iter(dataloader))
#         new_batch = move_to_device(new_batch, device)
#         # tmp_batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in tmp_batch.items()}
            
#         outputs, audio_len = model(**new_batch)
#         prompt_ids = new_batch['prompt_ids']
#         end_prompt_ids = new_batch['end_prompt_ids']
#         caption_ids = new_batch['caption_ids']
                    
#         prompt_ids_seq = prompt_ids.shape[1]
#         end_prompt_ids_seq = end_prompt_ids.shape[1]
#         audio_seq = audio_len
#         logits_start = prompt_ids_seq + audio_seq + end_prompt_ids_seq
            
#         logits = outputs.logits
#         op_logits = logits[:, logits_start:-1, :].contiguous()
#         caption_labels = caption_ids[:, 1:].contiguous()
            
#         sampled = torch.multinomial(op_logits[:, -1, :].softmax(dim=-1), caption_labels.shape[1])
            
#         print(f"ground truth caption: {tokenizer.batch_decode(new_batch['caption_ids'])}")
#         print(f"model outout caption: {tokenizer.batch_decode(sampled)}")

In [34]:
# tmp_batch = next(iter(val_dataloader))


In [35]:
# tmp_batch = move_to_device(batch, device)

In [36]:
# ops = model(**tmp_batch)


In [37]:
# results = run_inference(model, val_dataloader, tokenizer, device, 2)

In [38]:
# print(results[0])

In [39]:
# for i in range(0, len(tmp_batch['caption_ids'])):
#     print(len(tmp_batch['caption_ids'][i]))
# len(tmp_batch['caption_ids'])

In [40]:
# tokenizer.encode('<|eot_id|>')

In [41]:
# tokenizer.batch_decode(tmp_batch['caption_ids'], skip_special_tokens=True)

In [77]:
def log_results_to_wandb(results):
    table = wandb.Table(columns=["Ground Truth", "Generated"])
    for result in results:
        table.add_data(result["Ground Truth"], result["Generated"])
    wandb.log({"Caption Comparison": table})
    
def run_inference(model, dataloader, tokenizer, device, num_samples=10):
    model.eval()
    results = []
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader, desc="Running inference")):
            if i >= num_samples:
                break
            
            batch = move_to_device(batch, device)
            
            outputs, audio_len = model(**batch)
            prompt_ids = batch['prompt_ids']
            end_prompt_ids = batch['end_prompt_ids']
            caption_ids = batch['caption_ids']
            
            prompt_ids_seq = prompt_ids.shape[1]
            end_prompt_ids_seq = end_prompt_ids.shape[1]
            audio_seq = audio_len
            logits_start = prompt_ids_seq + audio_seq + end_prompt_ids_seq
            
            logits = outputs.logits
            op_logits = logits[:, logits_start:-1, :].contiguous()
            caption_labels = caption_ids[:, 1:].contiguous()
            
            sampled = torch.multinomial(op_logits[:, -1, :].softmax(dim=-1), caption_labels.shape[1])
            
            ground_truth = tokenizer.batch_decode(batch['caption_ids'], skip_special_tokens=True)
            generated = tokenizer.batch_decode(sampled,skip_special_tokens=True)
            
            for gt, gen in zip(ground_truth, generated):
                results.append({"Ground Truth": gt, "Generated": gen})
    
    return results

In [78]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments
from typing import Dict, List, Union
from tqdm import tqdm
class MultiModalDataset(Dataset):
    def __init__(self, dataloader):
        self.dataloader = dataloader
    
    def __len__(self):
        return len(self.dataloader.dataset)

    def __getitem__(self, idx):
        return self.dataloader.dataset[idx]

class MultiModalTrainer(Trainer):
    def __init__(self, test_dataloader, tokenizer, test_steps=500, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.test_dataloader = test_dataloader
        self.test_step = 0
        self.tokenizer = tokenizer
        self.test_steps = test_steps
        
    def move_to_device(self, batch, device):
        return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        
    def training_step(self, model, inputs):
        """
        Perform a training step and run test loop every 500 steps.
        """
        # Perform regular training step
        loss = super().training_step(model, inputs)

        # Increment test step counter
        self.test_step += 1

        # Run test loop every 500 steps
        # for test purposes logging at every step
        if self.test_step > 0 and self.test_step % self.test_steps == 0:
            self.run_test_loop()

        return loss
        
    def run_test_loop(self):
        inference_results = run_inference(self.model, 
                                          self.test_dataloader, 
                                          self.tokenizer, 
                                          device, 
                                          num_samples=2)
        log_results_to_wandb(inference_results)
        
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs, audio_seq = model(**inputs)
        loss = self.calculate_loss(outputs, inputs, audio_seq)
        return (loss, outputs) if return_outputs else loss

    def calculate_loss(self, outputs, batch, audio_seq):
        logits = outputs.logits
        prompt_ids = batch['prompt_ids']
        end_prompt_ids = batch['end_prompt_ids']
        caption_ids = batch['caption_ids']
        
        prompt_ids_seq = prompt_ids.shape[1]
        end_prompt_ids_seq = end_prompt_ids.shape[1]
        logits_start = prompt_ids_seq + audio_seq + end_prompt_ids_seq

        op_logits = logits[:, logits_start:-1, :].contiguous()
        caption_labels = caption_ids[:, 1:].contiguous()
        
        if op_logits.shape[1] != caption_labels.shape[1]:
            raise ValueError(f"Shape mismatch: op_logits {op_logits.shape}, caption_labels {caption_labels.shape}")

        loss = torch.nn.functional.cross_entropy(
            op_logits.view(-1, op_logits.shape[-1]), caption_labels.view(-1)
        )
        return loss

    def evaluate(
            self,
            eval_dataset = None,
            ignore_keys = None,
            metric_key_prefix: str = "eval",
        ):
            eval_dataloader = self.get_eval_dataloader(eval_dataset)
            # Perform decoding and loss calculations here
            # print(self.model)
            metrics = {'wer': 1.0}
            # Validation
            self.model.eval()
            total_val_loss = 0
            with torch.no_grad():
                for batch in tqdm(eval_dataloader, desc="Validating"):
                    batch = self.move_to_device(batch, device)
                    # batch = {k: v.to(device) for k, v in batch.items()}
                    outputs, audio_seq = model(**batch)
                    # loss = outputs.loss
                    # outputs = model(**batch)
                    loss = self.calculate_loss(outputs, batch, audio_seq)
                    # loss = calculate_loss(outputs, batch, audio_seq)
                    total_val_loss += loss.item()
                
            avg_val_loss = total_val_loss / len(eval_dataloader)
            print(f"Average Validation Loss: {avg_val_loss:.4f}")
            print()
        
            # Log metrics to wandb
            metrics = {
                "eval_loss": avg_val_loss
            }
            print(metrics)
            return metrics

# wandb.init(project="MultiModal LLM", name="HF Trainer Run")
# Prepare datasets
train_dataset = MultiModalDataset(train_dataloader)
eval_dataset = MultiModalDataset(val_dataloader)

In [26]:
model = MultiModalLLM(whisper_model_name, model_name)
model = move_model_to_gpu(model)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Define training arguments
wandb.init(project="MultiLM-testeval config", name="steps 2000 - eval 500- good-test")
training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=10,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=75,
        evaluation_strategy="steps",
        eval_steps=75,
        save_steps=1000,
        max_steps=1000,
        load_best_model_at_end=True,
        learning_rate=1.5e-3,
        report_to="wandb",
    )

# Initialize Trainer
trainer = MultiModalTrainer(
        model=model,
        args=training_args,
        data_collator=collate_fn,
        train_dataset=new_train_dataset,
        eval_dataset=new_val_dataset,
        test_dataloader=test_dataloader,
        tokenizer=tokenizer,
        test_steps=75
    )

# Train the model
trainer.train()

# # Save the final model
# trainer.save_model("final_model")

# # Finish the wandb run
wandb.finish()